**Zadanie 3. (5p)** Napisz program, który przeglądając 1-gramy i 2-gramy (nkjp ngrams) z języka
polskiego znajdzie możliwie najwięcej sytuacji, w których popełniono literówkę związaną z wstawieniem, bądź pominięciem spacji. Wystarczy, że znajdziesz w sumie około 10 tysięcy błędów, ale
postaraj się, by były one możliwie jak najbardziej wiarygodne. Przykładowo, dla zadania wstawiania
spacji:
- Powinieneś znaleźć takie przykłady jak: wielkiegopomorska, socjologiiuniwersytetu, otwartapracownia, przezgrzechy
- **Nie** powinieneś znajdować: antysystemową, supertygrysa, wewnątrzoddziałowego, wschodniokarpackiego

In [87]:
import csv
from collections import defaultdict

In [85]:
UNIGRAMS_FILE_PATH = 'data/1grams'
BIGRAMS_FILE_PATH = 'data/2grams'

BASE_FORMS_FILE_PATH = 'data/polimorfologik-2.1.txt'

In [95]:
with open(BASE_FORMS_FILE_PATH) as f:
    _BASE_FORMS = {
        word.lower(): base.lower()
        for base, word, *__ in csv.reader(f, delimiter=';')
    }

def base(word):
    return _BASE_FORMS.get(word.lower())

def is_correct(word):
    return base(word) is not None

In [103]:
_WORD_COUNTS = defaultdict(int)
with open(UNIGRAMS_FILE_PATH) as f:
    for line in f:
        count_raw, word = line.split()
        _WORD_COUNTS[base(word)] += int(count_raw)
    
def get_count(word):
    return _WORD_COUNTS.get(base(word), 0)

In [131]:
MIN_COUNT = 1000

def is_likely(word):
    return is_correct(word) and get_count(word) > MIN_COUNT

In [139]:
NO_SPACE_PREFIXES = {
    'anty',
    'przeciw',
    'super',
    'wewnątrz',
    'ogólno',
    'ponad',
    'auto',
    'pod',
    'poz',
    'pan',
    'cie',
    'lub',
    'nie',
    'kar',
    'sto',
    'nowo',
    'mało',
    # NUMERBS
    'jedno',
    'dwu',
    'trzy',
    'cztero',
    'pięcio',
    'sześcio',
    'siedmio',
    'ośmio',
    'dziewięcio',
    'dziesięcio',
    # DIRECTIONS
    'wschodnio',
    'zachodnio',
    'północno',
    'południowo',
}

In [140]:
def was_space_wrongly_omitted(word):
    if is_correct(word):
        return False
    try:
        prefix, suffix = next(
            (word[:i], word[i:])
            for i in range(1, len(word))
            if is_likely(word[:i]) and is_likely(word[i:])
        )
    except StopIteration:
        return False
    return all([
        len(prefix) >= 3,
        len(suffix) >= 3,
        len(prefix) >= len(word) / 3,
        len(suffix) >= len(word) / 3,
        prefix not in NO_SPACE_PREFIXES,
    ])

In [141]:
def was_space_wrongly_inserted(word1, word2):
    return (
        is_likely(word1 + word2)
        and (not is_likely(word1) or not is_likely(word2))
    )

In [142]:
is_likely('poz')

True

In [144]:
RESULTS1_FILE_PATH = 'results/3_1.txt'

with open(RESULTS1_FILE_PATH, 'w+') as res:
    with open(UNIGRAMS_FILE_PATH) as f:
        for line in f:
            _, word = line.split()
            if was_space_wrongly_omitted(word):
                res.write(f'{word}\n')

In [42]:
was_space_wrongly_omitted('krzysztof')

True

In [71]:
is_correct('vito')

False

In [83]:
WORD_COUNTS.get('morzem')

2

In [97]:
get_count('mama')

20975

In [98]:
base('mama')

'mama'

In [149]:
RESULTS2_FILE_PATH = 'results/3_2.txt'

with open(RESULTS2_FILE_PATH, 'w+') as res:
    with open(BIGRAMS_FILE_PATH) as f:
        for line in f:
            _, word1, word2 = line.split()
            if was_space_wrongly_inserted(word1, word2):
                res.write(f'{word1} {word2}\n')